In [1]:
# ! pip install pyoso

In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()

OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

# Step 1: Check the sheet is present

In [3]:
client.to_pandas("""
SELECT *
FROM oso_gsheets.default.op_atlas_test_1
LIMIT 5
""")

,project name,oso_slug,is mapped,season,status,initial delivery date,op delivered,op total amount,intent,proposal link,other metadata
0,Synthetix,synthetix,TRUE,Grants Season 1,Sent,2022-07-27,None,"9,000,000",<NA>,https://gov.optimism.io/t/gf-phase-0-proposal-...,"{""Incentive Program Launched?"": ""Yes"", ""Propos..."
1,Perpetual Protocol,perpetual-protocol,TRUE,Grants Season 1,Sent,2022-07-08,None,"9,000,000",<NA>,https://gov.optimism.io/t/gf-phase-0-proposal-...,"{""Incentive Program Launched?"": ""Yes"", ""Propos..."
2,Velodrome,velodrome,TRUE,Grants Season 2,Sent,<NA>,None,"4,000,000",<NA>,https://gov.optimism.io/t/ready-gf-phase-1-pro...,"{""Proposal Link"": ""https://gov.optimism.io/t/r..."
3,Lyra V1,derivexyz,TRUE,Grants Season 1,Sent,2022-07-20,None,"3,000,000",<NA>,https://gov.optimism.io/t/gf-phase-0-proposal-...,"{""Incentive Program Launched?"": ""Yes"", ""Propos..."
4,Grow the Superchain economy by incentivizing D...,base-org,TRUE,Grants Season 6,Passed,<NA>,None,"3,000,000",Intent 3B,https://app.charmverse.io/op-grants/grow-the-s...,"{""Incentive Program Launched?"": ""Builders"", ""P..."


# Step 2: Join sheet data on OSO data

In [4]:
df_projects = client.to_pandas("""
SELECT
 p.project_id,
 sheet."oso_slug",
 sheet."project name" AS application_name,
 sheet."season",
 sheet."op total amount" AS op_total_amount
FROM oso_gsheets.default.op_atlas_test_1 AS sheet
JOIN oso.projects_v1 p ON sheet.oso_slug = p.project_name
WHERE sheet.oso_slug IS NOT NULL
""")
df_projects.tail()

,project_id,oso_slug,application_name,season,op_total_amount
397,nR8cVZ4htmJwjpypWKOcoCslmzbg2zvvzWLRd3VRDwU=,geo-web,Geo Web,Grants Season 3,"2,000"
398,zakGuUr6Ayh94eSxMgAZmI9SfCRrm4MFH/7AfFVR0hA=,dspytdao,Microgrant: Pin Save,Grants Season 6,"2,000"
399,wl9UkM9CMabW8bjhs556F/pkniW7/uubFPdpAwNkVGE=,cyfrin,Curve Audit Grant Request (Cyfrin),Grants Season 6,"1,472"
400,M7mj4E6rVNxqX6O3QbNNaworQ4q/zn9J1XKeZ4Ch2lY=,aragon,Aragon <> RISC Zero - Joint partnership on cro...,Grants Season 6,"1,250"
401,zakGuUr6Ayh94eSxMgAZmI9SfCRrm4MFH/7AfFVR0hA=,dspytdao,Microgrant: Pin Save,Grants Season 6,500


# Step 3: Grab key metrics for projects

In [5]:
df_metrics = client.to_pandas("""
SELECT
 p.project_id,
 sheet."oso_slug",
 sheet."project name" AS application_name,
 sheet."season",
 sheet."op total amount" AS op_total_amount,
 m.display_name,
 km.amount
FROM oso_gsheets.default.op_atlas_test_1 AS sheet
JOIN oso.projects_v1 AS p ON sheet.oso_slug = p.project_name
JOIN oso.key_metrics_by_project_v0 AS km ON p.project_id = km.project_id
JOIN oso.metrics_v0 AS m ON km.metric_id = m.metric_id
WHERE
  sheet.oso_slug IS NOT NULL
  AND m.display_name IN ('Active Contracts', 'Repositories', 'Contributors', 'Commits', 'Contract Invocations', 'Gas Fees')
""")

df_metrics

,project_id,oso_slug,application_name,season,op_total_amount,display_name,amount
0,L1FGc2AIMWcbDulPkpgzndWnljElZEAxxr+jPOdn0WQ=,synthetix,Synthetix,Grants Season 1,"9,000,000",Contract Invocations,70217295.0
1,L1FGc2AIMWcbDulPkpgzndWnljElZEAxxr+jPOdn0WQ=,synthetix,Synthetix,Grants Season 1,"9,000,000",Commits,24901.0
2,L1FGc2AIMWcbDulPkpgzndWnljElZEAxxr+jPOdn0WQ=,synthetix,Synthetix,Grants Season 1,"9,000,000",Contract Invocations,2960398.0
3,L1FGc2AIMWcbDulPkpgzndWnljElZEAxxr+jPOdn0WQ=,synthetix,Synthetix,Grants Season 1,"9,000,000",Active Contracts,19.0
4,L1FGc2AIMWcbDulPkpgzndWnljElZEAxxr+jPOdn0WQ=,synthetix,Synthetix,Grants Season 1,"9,000,000",Gas Fees,73.612947
...,...,...,...,...,...,...,...
3418,zakGuUr6Ayh94eSxMgAZmI9SfCRrm4MFH/7AfFVR0hA=,dspytdao,Microgrant: Pin Save,Grants Season 6,500,Commits,3814.0
3419,zakGuUr6Ayh94eSxMgAZmI9SfCRrm4MFH/7AfFVR0hA=,dspytdao,Microgrant: Pin Save,Grants Season 6,500,Contributors,12.0
3420,zakGuUr6Ayh94eSxMgAZmI9SfCRrm4MFH/7AfFVR0hA=,dspytdao,Microgrant: Pin Save,Grants Season 6,500,Active Contracts,1.0
3421,zakGuUr6Ayh94eSxMgAZmI9SfCRrm4MFH/7AfFVR0hA=,dspytdao,Microgrant: Pin Save,Grants Season 6,500,Gas Fees,0.003455


In [6]:
df_pvt = df_metrics.pivot_table(
    index=['project_id', 'oso_slug', 'application_name', 'season', 'op_total_amount'],
    columns=['display_name'],
    values='amount',
    aggfunc='sum',
    fill_value=0,
)
df_pvt.sort_values(by='Gas Fees', ascending=False).head(10)

display_name                                                                                                                               Active Contracts  \
project_id                                   oso_slug           application_name                          season          op_total_amount                     
pWEQHj4IsxU8lx2EDLJpphYaGAvw/yRps3vgTbVCXjQ= uniswap            Uniswap: YBA Grant                        Grants Season 6 180,000                   33396.0   
                                                                Uniswap: LRT Grant                        Grants Season 6 120,000                   33396.0   
                                                                Uniswap on Superchain (OP, INK, Unichain) Grants Season 7 500,000                   33396.0   
                                                                Uniswap V3                                Grants Season 1 1,000,000                 33396.0   
ZtwRwweUfJxkM3rtVoIfvvOaqHRc7CGlSCAKE8VgmN0= aerodrome-finance  Aerodrome Finance                         Grants Season 7 650,000                    6043.0   
ziS9zVwL0wejwpRl3exXS/sPkVTxes5OeRkH1buQ4Y4= perpetual-protocol Perpetual Protocol                        Grants Season 1 9,000,000                   757.0   
L1FGc2AIMWcbDulPkpgzndWnljElZEAxxr+jPOdn0WQ= synthetix          Synthetix Mission Request for Perps DEX   Grants Season 5 300,000                    1025.0   
                                                                Synthetix                                 Grants Season 1 9,000,000                  1025.0   
                                                                Synthetix Growth Grant Request            Grants Season 5 200,000                    1025.0   
                                                                Synthetix Mission Request for Perps DEX   Grants Season 5 100,000                    1025.0   

display_name                                                                                                                               Commits  \
project_id                                   oso_slug           application_name                          season          op_total_amount            
pWEQHj4IsxU8lx2EDLJpphYaGAvw/yRps3vgTbVCXjQ= uniswap            Uniswap: YBA Grant                        Grants Season 6 180,000          13928.0   
                                                                Uniswap: LRT Grant                        Grants Season 6 120,000          13928.0   
                                                                Uniswap on Superchain (OP, INK, Unichain) Grants Season 7 500,000          13928.0   
                                                                Uniswap V3                                Grants Season 1 1,000,000        13928.0   
ZtwRwweUfJxkM3rtVoIfvvOaqHRc7CGlSCAKE8VgmN0= aerodrome-finance  Aerodrome Finance                         Grants Season 7 650,000             85.0   
ziS9zVwL0wejwpRl3exXS/sPkVTxes5OeRkH1buQ4Y4= perpetual-protocol Perpetual Protocol                        Grants Season 1 9,000,000         1675.0   
L1FGc2AIMWcbDulPkpgzndWnljElZEAxxr+jPOdn0WQ= synthetix          Synthetix Mission Request for Perps DEX   Grants Season 5 300,000          24901.0   
                                                                Synthetix                                 Grants Season 1 9,000,000        24901.0   
                                                                Synthetix Growth Grant Request            Grants Season 5 200,000          24901.0   
                                                                Synthetix Mission Request for Perps DEX   Grants Season 5 100,000          24901.0   

display_name                                                                                                                               Contract Invocations  \
project_id                                   oso_slug           application_name                          season          op_to